In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from glob import glob
import os
from matplotlib.patches import Circle

: 

Configuration

Select the version, to visualize (Just uncomment the one and comment the others)

In [ ]:

# VERSION = 'serial'  # Filename pattern: fhp_00100.csv
# VERSION = 'mpi'     # Filename pattern: fhp_00100_rank0.csv, fhp_00100_rank1.csv, ...
# VERSION = 'cuda'    # Filename pattern: fhp_cuda_00100.csv

In [ ]:
DATA_FOLDER = 'output'       # Folder containing your CSV files
BLOCK_SIZE = 6               # Size of "Macro-cell" for averaging (e.g., 6x6 pixels)
FPS = 10                     # Animation speed

# Simulation Dimensions (Must match C++ constants)
NX_RAW = 300
NY_RAW = 100

# Obstacle Info (Must match C++ constants)
OBSTACLE_X = 150
OBSTACLE_Y = 50
OBSTACLE_R = 20

Data Processing Function

In [ ]:
def get_file_pattern(step_num):
    """
    Returns the glob pattern for a specific timestep based on VERSION.
    """
    if VERSION == 'serial':
        return os.path.join(DATA_FOLDER, f"fhp_{step_num:05d}.csv")
    elif VERSION == 'mpi':
        return os.path.join(DATA_FOLDER, f"fhp_{step_num:05d}_rank*.csv")
    elif VERSION == 'cuda':
        return os.path.join(DATA_FOLDER, f"fhp_cuda_{step_num:05d}.csv")
    else:
        raise ValueError(f"Unknown VERSION: {VERSION}. Use 'serial', 'mpi', or 'cuda'.")

In [ ]:
def load_and_process_frame(step_num, block_size=6):
    """
    Loads raw FHP particle data for a specific timestep and performs 
    coarse-graining (spatial averaging) to smooth out the noise.
    
    Works for Serial, MPI (multiple rank files), and CUDA versions.
    """
    # Get the file pattern
    pattern = get_file_pattern(step_num)
    files = sorted(glob(pattern))
    
    if not files:
        print(f"Warning: No files found for step {step_num} with pattern {pattern}")
        return None
    
    # Load and concatenate (handles MPI's multiple files automatically)
    dfs = [pd.read_csv(f) for f in files]
    df = pd.concat(dfs, ignore_index=True)
    
    # Sort by coordinates
    df = df.sort_values(['y', 'x'])
    
    # Convert to 2D Grids
    u_raw = df.pivot(index='y', columns='x', values='u_avg').values
    v_raw = df.pivot(index='y', columns='x', values='v_avg').values
    rho_raw = df.pivot(index='y', columns='x', values='density').values
    
    # Coarse Graining
    ny_coarse = NY_RAW // block_size
    nx_coarse = NX_RAW // block_size
    
    def coarse_grain(arr):
        arr_trim = arr[:ny_coarse*block_size, :nx_coarse*block_size]
        return arr_trim.reshape(ny_coarse, block_size, nx_coarse, block_size).mean(axis=(1, 3))

    u_coarse = coarse_grain(u_raw)
    v_coarse = coarse_grain(v_raw)
    rho_coarse = coarse_grain(rho_raw)
    
    # Compute Derived Quantities
    speed = np.sqrt(u_coarse**2 + v_coarse**2)
    
    # Vorticity (Curl of velocity field: dv/dx - du/dy)
    dv_dx = np.gradient(v_coarse, axis=1)
    du_dy = np.gradient(u_coarse, axis=0)
    vorticity = dv_dx - du_dy
    
    # Coordinate Mesh for Plotting
    x = np.linspace(0, NX_RAW, nx_coarse)
    y = np.linspace(0, NY_RAW, ny_coarse)
    X, Y = np.meshgrid(x, y)
    
    return X, Y, u_coarse, v_coarse, speed, vorticity

Get Available Timesteps

In [ ]:
def get_available_steps():
    """
    Scans the output folder and returns a sorted list of available timesteps.
    """
    if VERSION == 'serial':
        pattern = os.path.join(DATA_FOLDER, "fhp_*.csv")
    elif VERSION == 'mpi':
        pattern = os.path.join(DATA_FOLDER, "fhp_*_rank0.csv")  # Use rank0 as reference
    elif VERSION == 'cuda':
        pattern = os.path.join(DATA_FOLDER, "fhp_cuda_*.csv")
    
    files = sorted(glob(pattern))
    steps = [extract_step_from_filename(f) for f in files]
    return sorted(set(steps))  # Remove duplicates and sort

Plotting Function

In [ ]:
def plot_single_frame(step_num=None):
    """
    Plots a single frame with streamlines.
    If step_num is None, plots the last available timestep.
    """
    available_steps = get_available_steps()
    
    if not available_steps:
        print(f"No data found in {DATA_FOLDER} for VERSION='{VERSION}'")
        return
    
    if step_num is None:
        step_num = available_steps[-1]  # Use last timestep
    
    if step_num not in available_steps:
        print(f"Step {step_num} not found. Available: {available_steps}")
        return
    
    print(f"Processing Step {step_num} (VERSION: {VERSION})...")
    result = load_and_process_frame(step_num, BLOCK_SIZE)
    
    if result is None:
        return
    
    X, Y, U, V, Speed, Vort = result
    
    # Setup Figure
    fig, ax = plt.subplots(figsize=(12, 5))
    
    # Plot Speed as Background with Streamlines
    st = ax.streamplot(X, Y, U, V, color=Speed, cmap='inferno', 
                       density=1.5, linewidth=1, arrowsize=1)
    
    # Add Obstacle
    circle = Circle((OBSTACLE_X, OBSTACLE_Y), OBSTACLE_R, color='gray', zorder=10)
    ax.add_patch(circle)
    
    # Formatting
    ax.set_xlim(0, NX_RAW)
    ax.set_ylim(0, NY_RAW)
    ax.set_aspect('equal')
    ax.set_title(f"FHP Flow Field ({VERSION.upper()}) - Step {step_num}")
    fig.colorbar(st.lines, ax=ax, label="Velocity Magnitude")
    plt.tight_layout()
    plt.show()

Animation Generator

In [ ]:
def create_animation():
    """
    Creates an animation of the flow evolution.
    """
    available_steps = get_available_steps()
    
    if not available_steps:
        print("No data found.")
        return
    
    print(f"Creating animation from {len(available_steps)} timesteps...")
    
    fig, ax = plt.subplots(figsize=(12, 5))
    
    def update(frame_idx):
        ax.clear()
        step_num = available_steps[frame_idx]
        
        result = load_and_process_frame(step_num, BLOCK_SIZE)
        if result is None:
            return ax,
        
        X, Y, U, V, Speed, Vort = result
        
        # Plot Vorticity Map (Red = CW, Blue = CCW)
        im = ax.imshow(Vort, cmap='bwr', origin='lower', 
                       extent=[0, NX_RAW, 0, NY_RAW], vmin=-0.2, vmax=0.2)
        
        # Overlay sparse velocity vectors
        skip = 2
        ax.quiver(X[::skip, ::skip], Y[::skip, ::skip], 
                  U[::skip, ::skip], V[::skip, ::skip], 
                  color='k', alpha=0.3, scale=30)
        
        # Obstacle
        circle = Circle((OBSTACLE_X, OBSTACLE_Y), OBSTACLE_R, color='black', zorder=10)
        ax.add_patch(circle)
        
        ax.set_title(f"Vorticity & Flow ({VERSION.upper()}) - Step {step_num}")
        ax.set_xlim(0, NX_RAW)
        ax.set_ylim(0, NY_RAW)
        return ax,

    ani = animation.FuncAnimation(fig, update, frames=len(available_steps), 
                                  interval=1000/FPS, blit=False)
    
    # Save
    output_file = f'flow_animation_{VERSION}.mp4'
    ani.save(output_file, writer='ffmpeg', dpi=150)
    print(f"Animation saved to {output_file}")
    plt.close()

Execution

In [ ]:
# 1. Plot the final state to check results
plot_single_frame()  # Automatically uses last timestep

# 2. Uncomment below to generate a full video (takes longer)
# create_animation()

# 3. Or plot a specific timestep:
# plot_single_frame(step_num=1000)